<a href="https://colab.research.google.com/github/nan12191/iPhone-dataset-analysis/blob/main/iPhone_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Preprocessing

In [12]:
from google.colab import files
files.upload()  # Choose the kaggle.json file from your local machine

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nanwang12191219","key":"e15bca26617cc659945416ca296ba5b5"}'}

In [13]:
!pip install kaggle

In [14]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [15]:
!kaggle datasets download -d mrmars1010/iphone-customer-reviews-nlp

Dataset URL: https://www.kaggle.com/datasets/mrmars1010/iphone-customer-reviews-nlp
License(s): CC0-1.0
iphone-customer-reviews-nlp.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
!unzip iphone-customer-reviews-nlp.zip

Archive:  iphone-customer-reviews-nlp.zip
replace iphone.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [18]:
!ls

 iphone.csv   iphone-customer-reviews-nlp.zip  'kaggle (1).json'   sample_data


In [24]:
import pandas as pd

df = pd.read_csv('iphone.csv')
print(df.head())
print(df.shape)

  productAsin country        date  isVerified  ratingScore  \
0  B09G9BL5CP   India  11-08-2024        True            4   
1  B09G9BL5CP   India  16-08-2024        True            5   
2  B09G9BL5CP   India  14-05-2024        True            4   
3  B09G9BL5CP   India  24-06-2024        True            5   
4  B09G9BL5CP   India  18-05-2024        True            5   

               reviewTitle                                  reviewDescription  \
0               No charger  Every thing is good about iPhones, there's not...   
1          iPhone 13 256GB  It look so fabulous, I am android user switche...   
2  Flip camera option nill  I tried to flip camera while recording but no ...   
3                  Product                                       100% genuine   
4             Good product         Happy to get the iPhone 13 in Amazon offer   

                                           reviewUrl  \
0  https://www.amazon.in/gp/customer-reviews/R345...   
1  https://www.amazon.in/gp/

### 1. Word2Vec

Using Word2Vec to find

In [27]:
word2vecdf = df.copy()

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

nltk.download('punkt')
nltk.download('stopwords')

# Clean and tokenize reviews
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize and remove stop words and punctuation
    tokens = word_tokenize(text.lower())
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

df['tokenized_reviews'] = df['reviewDescription'].apply(preprocess_text)

from gensim.models import Word2Vec

# Train Word2Vec model
model = Word2Vec(df['tokenized_reviews'], vector_size=100, window=5, min_count=2, workers=4)

# Save the trained model for later use
model.save("word2vec.model")

import numpy as np

def average_word_vectors(words, model, vector_size):
    valid_words = [word for word in words if word in model.wv]
    if len(valid_words) == 0:
        return np.zeros(vector_size)
    return np.mean([model.wv[word] for word in valid_words], axis=0)

# Compute the review vectors
df['review_vector'] = df['tokenized_reviews'].apply(lambda x: average_word_vectors(x, model, 100))

from sklearn.cluster import KMeans

# Convert the review vectors to a format suitable for clustering
X = np.array(df['review_vector'].tolist())

# Apply K-means clustering
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Analyze cluster sizes
print(df.groupby('cluster').size())


# Print some reviews from each cluster
for cluster in range(5):
    print(f"\nCluster {cluster}:")
    print(df[df['cluster'] == cluster]['review_column'].head())


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: 'float' object has no attribute 'lower'

### 2. Word2Vec

### 2. Word2Vec

### 2. Word2Vec